## 資料集說明 :
- 來源 : [kaggle](https://www.kaggle.com/carrie1/ecommerce-data?select=data.csv)
- 資料集 : 在英國註冊的線上零售商店
- 資料集大小 : 541909 筆資料，8個欄位
- 時間 : 2010/01/12 ~ 2011/09/12 
- 其他 : 該公司主要銷售各種場合的獨特禮品，公司的許多客戶都是批發商。

## 欄位說明 :
- InvoiceNo : 發票編號
- StockCode : 股票編號
- Description : 商品敘述
- Quantity : 數量
- InvoiceDate : 發票日期
- UnitPrice : 單價
- (主鍵) CustomerID : 顧客ID
- Country : 國家

## 預期做法 :
1. 顧客 RFM 分析並分群預測回購金額

In [1]:
pacman::p_load("tidyverse", "data.table")

df = fread("./dataset/data.csv", fill = T, stringsAsFactor = F)
cat("Dataframe dimensions: ", dim(df))
head(df)
glimpse(df)

Dataframe dimensions:  541909 8

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850,United Kingdom


Observations: 541,909
Variables: 8
$ InvoiceNo   <chr> "536365", "536365", "536365", "536365", "536365", "5363...
$ StockCode   <chr> "85123A", "71053", "84406B", "84029G", "84029E", "22752...
$ Description <chr> "WHITE HANGING HEART T-LIGHT HOLDER", "WHITE METAL LANT...
$ Quantity    <int> 6, 6, 8, 6, 6, 2, 6, 6, 6, 32, 6, 6, 8, 6, 6, 3, 2, 3, ...
$ InvoiceDate <chr> "12/1/2010 8:26", "12/1/2010 8:26", "12/1/2010 8:26", "...
$ UnitPrice   <dbl> 2.55, 3.39, 2.75, 3.39, 3.39, 7.65, 4.25, 1.85, 1.85, 1...
$ CustomerID  <int> 17850, 17850, 17850, 17850, 17850, 17850, 17850, 17850,...
$ Country     <chr> "United Kingdom", "United Kingdom", "United Kingdom", "...


In [2]:
df$CustomerID = as.character(df$CustomerID) # int to string
df$InvoiceDate = as.Date(df$InvoiceDate, format = "%m/%d/%Y") # char to Date

Observations: 541,909
Variables: 8
$ InvoiceNo   <chr> "536365", "536365", "536365", "536365", "536365", "5363...
$ StockCode   <chr> "85123A", "71053", "84406B", "84029G", "84029E", "22752...
$ Description <chr> "WHITE HANGING HEART T-LIGHT HOLDER", "WHITE METAL LANT...
$ Quantity    <int> 6, 6, 8, 6, 6, 2, 6, 6, 6, 32, 6, 6, 8, 6, 6, 3, 2, 3, ...
$ InvoiceDate <date> 2010-12-01, 2010-12-01, 2010-12-01, 2010-12-01, 2010-1...
$ UnitPrice   <dbl> 2.55, 3.39, 2.75, 3.39, 3.39, 7.65, 4.25, 1.85, 1.85, 1...
$ CustomerID  <chr> "17850", "17850", "17850", "17850", "17850", "17850", "...
$ Country     <chr> "United Kingdom", "United Kingdom", "United Kingdom", "...


In [7]:
sapply(df, function(x) sum(ifelse(is.na(x), 1, 0)))
df = na.omit(df) # remove NA
sapply(df, function(x) sum(ifelse(is.na(x), 1, 0)))

InvoiceNo   StockCode Description    Quantity InvoiceDate   UnitPrice 
          0           0           0           0           0           0 
 CustomerID     Country 
     135080           0

InvoiceNo   StockCode Description    Quantity InvoiceDate   UnitPrice 
          0           0           0           0           0           0 
 CustomerID     Country 
          0           0

In [16]:
summary(df) # Quantity 有負值，應該是退貨顧客

  InvoiceNo          StockCode         Description           Quantity        
 Length:406829      Length:406829      Length:406829      Min.   :-80995.00  
 Class :character   Class :character   Class :character   1st Qu.:     2.00  
 Mode  :character   Mode  :character   Mode  :character   Median :     5.00  
                                                          Mean   :    12.06  
                                                          3rd Qu.:    12.00  
                                                          Max.   : 80995.00  
  InvoiceDate           UnitPrice         CustomerID          Country         
 Min.   :2010-12-01   Min.   :    0.00   Length:406829      Length:406829     
 1st Qu.:2011-04-06   1st Qu.:    1.25   Class :character   Class :character  
 Median :2011-07-31   Median :    1.95   Mode  :character   Mode  :character  
 Mean   :2011-07-10   Mean   :    3.46                                        
 3rd Qu.:2011-10-20   3rd Qu.:    3.75                     

In [30]:
df %>% count(Country, sort = T) %>% head() # 英國顧客佔最多
df %>% count(CustomerID, sort = T) %>% head() # 編號 17841 顧客買最多

Country,n
United Kingdom,361878
Germany,9495
France,8491
EIRE,7485
Spain,2533
Netherlands,2371


CustomerID,n
17841,7983
14911,5903
14096,5128
12748,4642
14606,2782
15311,2491


In [60]:
df$tid = group_indices(df, InvoiceDate, CustomerID) # 將顧客同天購買的商品歸為一張訂單